![CoSAppLogo](../images/cosapp.svg)

<font color='orange'>**CoSApp**</font> tutorials on multimode systems

# Multimode ODE

In this example, we show a system defining a simple Ordinary Differential Equation, with an event-driven discontinuity.

In [ ]:
from cosapp.base import System


class ScalarOde(System):
    """System representing ODE df/dt = F(t)"""
    def setup(self):
        self.add_inward('df')
        self.add_transient('f', der='df')


class MultimodeScalarOde(System):
    def setup(self):
        self.add_child(ScalarOde('ode'), pulling=['f', 'df'])
        self.add_outward_modevar('snapped', False)
        self.add_event('snap')  # event defining mode var `snapped`
    
    def transition(self):
        if self.snap.present:
            self.snapped = True


In [ ]:
from cosapp.drivers import RungeKutta
from cosapp.recorders import DataFrameRecorder

s = MultimodeScalarOde('s')

s.snap.trigger = "f > 0.547"

driver = s.add_driver(
    RungeKutta(order=2, time_interval=(0, 1), dt=0.1)
)
driver.add_recorder(DataFrameRecorder(includes=['f', 'df', 'snapped']), period=0.1)

# Set scenario such that df/dt changes with mode
driver.set_scenario(
    init = {'f': 0},
    values = {'df': '2 * t if not snapped else 0'},
)
s.run_drivers()

# Retrieve recorded data
data = driver.recorder.export_data()
data = data.drop(['Section', 'Status', 'Error code'], axis=1)

In [ ]:
# Plot results
import plotly.graph_objs as go

def get_trace(col):
    return go.Scatter(
        x = data['time'],
        y = data[col],
        name = col,
        mode = "lines+markers",
    )

go.Figure(
    data = [
        get_trace('f'),
        get_trace('df'),
    ],
    layout = go.Layout(
        xaxis = dict(title="Time"),
        height = 450,
        hovermode = "x",
    ),
)

### Show recorded data

Note the double entry at snap time.
When `snapped` is `False`, the solution is $f(t) = t^2$, since the derivative is $2t$, with initial condition $f(0) = 0$.
Event time is therefore $t_{\rm snap} = \sqrt{0.547}$.

In [ ]:
from math import sqrt
print(f"{sqrt(0.547) = }")

In [ ]:
data